# Imports

In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import vstack

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

# Download and read data

In [2]:
market = pd.read_csv('../data/estaticos_market.zip', index_col='Unnamed: 0')
port_1 = pd.read_csv('../data/estaticos_portfolio1.csv', index_col='Unnamed: 0', usecols=['Unnamed: 0', 'id'])
port_2 = pd.read_csv('../data/estaticos_portfolio2.csv', index_col='Unnamed: 0', usecols=['Unnamed: 0', 'id'])
port_3 = pd.read_csv('../data/estaticos_portfolio3.csv', index_col='Unnamed: 0', usecols=['Unnamed: 0', 'id'])

# Clean data

In [3]:
missing = (market.isna().mean() > 0.5)

In [4]:
market = market.loc[:, ~missing]

In [5]:
market.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 462298 entries, 0 to 462297
Data columns (total 50 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   id                                     462298 non-null  object 
 1   fl_matriz                              462298 non-null  bool   
 2   de_natureza_juridica                   462298 non-null  object 
 3   sg_uf                                  462298 non-null  object 
 4   natureza_juridica_macro                462298 non-null  object 
 5   de_ramo                                462298 non-null  object 
 6   setor                                  460371 non-null  object 
 7   idade_empresa_anos                     462298 non-null  float64
 8   idade_emp_cat                          462298 non-null  object 
 9   fl_me                                  462298 non-null  bool   
 10  fl_sa                                  462298 non-null  

In [6]:
related_cols = [
    ('idade_emp_cat', 'idade_empresa_anos'),
    ('fl_st_especial', 'dt_situacao'),
    ('fl_veiculo', ['vl_total_veiculos_pesados_grupo', 'vl_total_veiculos_leves_grupo']),
    ('idade_media_socios', ['idade_maxima_socios', 'idade_minima_socios']),
    ('de_faixa_faturamento_estimado', 'vl_faturamento_estimado_aux'),
    ('de_faixa_faturamento_estimado_grupo', 'vl_faturamento_estimado_grupo_aux')
]

In [7]:
to_remove = [
    'fl_telefone', 'fl_email', 'de_saude_rescencia', 'nu_meses_rescencia'
]

In [8]:
for _, c in related_cols:
    if isinstance(c, str):
        to_remove.extend([c])
    else:
        to_remove.extend(c)
to_remove

['fl_telefone',
 'fl_email',
 'de_saude_rescencia',
 'nu_meses_rescencia',
 'idade_empresa_anos',
 'dt_situacao',
 'vl_total_veiculos_pesados_grupo',
 'vl_total_veiculos_leves_grupo',
 'idade_maxima_socios',
 'idade_minima_socios',
 'vl_faturamento_estimado_aux',
 'vl_faturamento_estimado_grupo_aux']

In [9]:
market.drop(columns=to_remove, inplace=True)

# Preprocess data

In [10]:
categoric_cols = market.drop(columns='id').select_dtypes(include=[object, bool]).columns.to_list()
numeric_cols = market.drop(columns='id').select_dtypes(exclude=[object, bool]).columns.to_list()

In [11]:
for c in categoric_cols:
    print(market[c].value_counts())
    print('='*50)

True     433232
False     29066
Name: fl_matriz, dtype: int64
EMPRESARIO INDIVIDUAL                                                     295756
SOCIEDADE EMPRESARIA LIMITADA                                              91231
ASSOCIACAO PRIVADA                                                         29694
EMPRESA INDIVIDUAL DE RESPONSABILIDADE LIMITADA DE NATUREZA EMPRESARIA     15183
ORGAO DE DIRECAO LOCAL DE PARTIDO POLITICO                                  7301
                                                                           ...  
CONSORCIO PUBLICO DE DIREITO PRIVADO                                           1
ORGAO DE DIRECAO NACIONAL DE PARTIDO POLITICO                                  1
CONSORCIO DE EMPREGADORES                                                      1
SOCIEDADE SIMPLES EM NOME COLETIVO                                             1
FUNDO PRIVADO                                                                  1
Name: de_natureza_juridica, Length: 67, dtype: 

In [12]:
market['fl_rm'].replace({'SIM': True, 'NAO': False}, inplace=True)

In [13]:
legal_name = ['fl_me','fl_sa', 'fl_epp', 'fl_mei', 'fl_ltda']
market['legal_name'] = market[legal_name].idxmax(1)
market.drop(columns=legal_name, inplace=True)

In [14]:
for c in legal_name:
    categoric_cols.remove(c)
categoric_cols.append('legal_name')

In [15]:
market[numeric_cols].describe()

,empsetorcensitariofaixarendapopulacao,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,qt_socios_st_regular,qt_filiais
count,318469.000000,347207.000000,347207.000000,347207.000000,310696.000000,307381.000000,462298.000000
mean,1308.005725,1.496326,1.476681,0.019645,42.816452,1.396082,28.737044
std,1161.889222,3.276626,3.258079,0.195166,12.626447,2.578793,468.626094
min,100.000000,1.000000,0.000000,0.000000,-2.000000,1.000000,0.000000
25%,673.230000,1.000000,1.000000,0.000000,33.000000,1.000000,0.000000
50%,946.680000,1.000000,1.000000,0.000000,42.000000,1.000000,0.000000
75%,1518.080000,2.000000,2.000000,0.000000,51.000000,1.000000,0.000000
max,75093.840000,246.000000,246.000000,13.000000,127.000000,179.000000,9647.000000


In [16]:
market[numeric_cols] = market[numeric_cols].fillna(-1).astype(int)

In [17]:
market['empsetorcensitariofaixarendapopulacao'] = pd.cut(
    market['empsetorcensitariofaixarendapopulacao'], 
    [-2, 1, 500, 750, 1000, 1500, 2000, 3000, 4000, 5000, 10000, np.inf]
)

market['qt_socios'] = pd.cut(market['qt_socios'], [-2, 0, 1, 5, 15, 50, 100, np.inf])
market['qt_socios_pf'] = pd.cut(market['qt_socios_pf'], [-2, 0, 1, 5, 15, 50, 100, np.inf])
market['qt_socios_pj'] = pd.cut(market['qt_socios_pj'], [-2, 0, 1, np.inf])
market['qt_socios_st_regular'] = pd.cut(market['qt_socios_st_regular'], [-2, 0, 1, 5, 15, 50, 100, np.inf])

market['idade_media_socios'] = pd.cut(market['idade_media_socios'], [-2, 0, 22, 45, 65, np.inf])

market['qt_filiais'] = pd.cut(market['qt_filiais'], [-2, 0, 5, 10, 50, 100, 1000, np.inf])

In [18]:
market[numeric_cols] = market[numeric_cols].astype(str)

In [19]:
market[categoric_cols] = market[categoric_cols].fillna('SEM INFORMACAO').astype(str)

In [20]:
for c in categoric_cols + numeric_cols:
    market[c] = market[c].str.replace('\W', '_')

In [21]:
market[numeric_cols].head()

,empsetorcensitariofaixarendapopulacao,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,qt_socios_st_regular,qt_filiais
0,__2_0__1_0_,_1_0__5_0_,_1_0__5_0_,__2_0__0_0_,_22_0__45_0_,_1_0__5_0_,__2_0__0_0_
1,_1_0__500_0_,_0_0__1_0_,_0_0__1_0_,__2_0__0_0_,_22_0__45_0_,_0_0__1_0_,__2_0__0_0_
2,_500_0__750_0_,_0_0__1_0_,_0_0__1_0_,__2_0__0_0_,_22_0__45_0_,__2_0__0_0_,__2_0__0_0_
3,_1_0__500_0_,_0_0__1_0_,_0_0__1_0_,__2_0__0_0_,_22_0__45_0_,_0_0__1_0_,__2_0__0_0_
4,_2000_0__3000_0_,_0_0__1_0_,_0_0__1_0_,__2_0__0_0_,__2_0__0_0_,__2_0__0_0_,__2_0__0_0_


In [22]:
market[categoric_cols].head()

,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_emp_cat,fl_st_especial,fl_rm,nm_divisao,...,sg_uf_matriz,de_saude_tributaria,de_nivel_atividade,fl_simples_irregular,nm_meso_regiao,nm_micro_regiao,fl_passivel_iss,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo,legal_name
0,True,SOCIEDADE_EMPRESARIA_LIMITADA,RN,ENTIDADES_EMPRESARIAIS,INDUSTRIA_DA_CONSTRUCAO,CONSTRUÇÃO_CIVIL,10_a_15,False,True,CONSTRUCAO_DE_EDIFICIOS,...,RN,VERDE,ALTA,False,LESTE_POTIGUAR,NATAL,True,DE_R__1_500_000_01_A_R__4_800_000_00,DE_R__1_500_000_01_A_R__4_800_000_00,fl_me
1,True,EMPRESARIO_INDIVIDUAL,PI,OUTROS,SERVICOS_DE_ALOJAMENTO_ALIMENTACAO,SERVIÇO,1_a_5,False,True,ALIMENTACAO,...,PI,CINZA,BAIXA,False,CENTRO_NORTE_PIAUIENSE,TERESINA,True,DE_R__81_000_01_A_R__360_000_00,DE_R__81_000_01_A_R__360_000_00,fl_mei
2,True,EMPRESARIO_INDIVIDUAL,AM,OUTROS,TRANSPORTE__ARMAZENAGEM_E_CORREIO,SERVIÇO,5_a_10,False,True,TRANSPORTE_TERRESTRE,...,AM,AMARELO,MEDIA,False,CENTRO_AMAZONENSE,MANAUS,True,ATE_R__81_000_00,ATE_R__81_000_00,fl_mei
3,True,EMPRESARIO_INDIVIDUAL,AM,OUTROS,SERVICOS_DIVERSOS,SERVIÇO,5_a_10,False,True,REPARACAO_E_MANUTENCAO_DE_EQUIPAMENTOS_DE_INFO...,...,AM,AMARELO,MEDIA,False,CENTRO_AMAZONENSE,MANAUS,True,DE_R__81_000_01_A_R__360_000_00,DE_R__81_000_01_A_R__360_000_00,fl_me
4,True,EMPRESARIO_INDIVIDUAL,RN,OUTROS,SERVICOS_PROFISSIONAIS__TECNICOS_E_CIENTIFICOS,SERVIÇO,1_a_5,False,True,SERVICOS_DE_ARQUITETURA_E_ENGENHARIA_TESTES_E_...,...,RN,VERDE,ALTA,False,LESTE_POTIGUAR,NATAL,True,DE_R__81_000_01_A_R__360_000_00,DE_R__81_000_01_A_R__360_000_00,fl_me


# Create feature for similarity serch

In [23]:
def columns_to_string(x):
    return ' '.join(['%s_%s' % a for a in list(zip(x.index, x.values))])

In [24]:
market['content'] = market.drop(columns='id').apply(lambda x: columns_to_string(x), axis=1)

In [25]:
market['content'].head()

0    fl_matriz_True de_natureza_juridica_SOCIEDADE_...
1    fl_matriz_True de_natureza_juridica_EMPRESARIO...
2    fl_matriz_True de_natureza_juridica_EMPRESARIO...
3    fl_matriz_True de_natureza_juridica_EMPRESARIO...
4    fl_matriz_True de_natureza_juridica_EMPRESARIO...
Name: content, dtype: object

# Recomender model

In [26]:
item_ids = market['id'].to_list()

In [27]:
def get_item_profile(item_id):
    idx = item_ids.index(item_id)
    item_profile = tfidf_matrix[idx:idx+1]
    return item_profile

In [28]:
def get_item_profiles(ids):
    item_profiles_list = [get_item_profile(x) for x in ids]
    item_profiles = vstack(item_profiles_list)
    return item_profiles

In [29]:
def cosine_similarity_batch(profile, tfidf_matrix, chunk=64):
    batches = (profile.shape[0] // chunk) + 1
    similarity = [
        cosine_similarity(profile[(chunk * batch): (chunk * (batch + 1))], tfidf_matrix) 
        for batch in range(batches)
    ]
    return np.concatenate(similarity)

In [30]:
def recommend_items(ids, topn):
    profile = get_item_profiles(ids)
    similarity = cosine_similarity_batch(profile, tfidf_matrix, 64)
    similarity = similarity.mean(axis=0)
    recommendation = pd.DataFrame(data={'id':item_ids, 'similarity': similarity})
    recommendation = recommendation[~recommendation['id'].isin(ids)]
    recommendation = recommendation.sort_values(by='similarity', ascending=False)
    return recommendation.head(topn)

In [31]:
def hit_and_miss(rec, lst):
    hit = rec['id'].isin(lst).sum()
    return hit / len(lst), hit, len(lst)

In [32]:
vectorizer = TfidfVectorizer(dtype=np.float32)
tfidf_matrix = vectorizer.fit_transform(market['content'])

# Evaluate model

In [34]:
for i, port in enumerate([port_1, port_2, port_3]):
    port = port['id'].to_list()
    x, y = train_test_split(port, test_size=0.3, random_state=42)
    rec = recommend_items(x, topn=len(y) * 10)
    p_hits, n_hits, s_lst = hit_and_miss(rec, y)
    print('Portifólio {}'.format(i + 1))
    print('({:.2f} %) {} hits from {} in {} predictions'.format(p_hits*100, n_hits, s_lst, len(rec)))

Portifólio 1
(0.00 %) 0 hits from 167 in 1670 predictions
Portifólio 2
(74.71 %) 127 hits from 170 in 1700 predictions
Portifólio 3
(55.00 %) 44 hits from 80 in 800 predictions
